In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

import factors

In [2]:
def plot_macro_rolling(manager_returns, macro_returns, rolling=120, savefig=False):
    
    stats = []
    dates = []

    for i in np.arange(manager_returns.shape[0] - rolling + 1):

        manager_returns_rolling = manager_returns.iloc[i : rolling + i]
        macro_returns_rolling = macro_returns.iloc[i : rolling + i]

        date = manager_returns_rolling.index[-1]

        macro_returns_rolling = sm.tools.add_constant(macro_returns_rolling)
        model = sm.OLS(manager_returns_rolling, macro_returns_rolling).fit(cov_type='HAC', cov_kwds={'maxlags':12})

        summary = model.summary2(xname='Alpha MKT INT CRD CUR'.split()).tables[1]

        mkt = [summary.iloc[1][0], summary.iloc[1][0] + summary.iloc[1][1], summary.iloc[1][0] - summary.iloc[1][1]]
        intr = [summary.iloc[2][0], summary.iloc[2][0] + summary.iloc[2][1], summary.iloc[2][0] - summary.iloc[2][1]]
        crd = [summary.iloc[3][0], summary.iloc[3][0] + summary.iloc[3][1], summary.iloc[3][0] - summary.iloc[3][1]]
        cur = [summary.iloc[4][0], summary.iloc[4][0] + summary.iloc[4][1], summary.iloc[4][0] - summary.iloc[4][1]]


        stats.append([mkt, intr, crd, cur])
        dates.append(date)



    fig, ((ax1, ax2),(ax3, ax4)) = plt.subplots(2,2)
    fig.set_size_inches(15, 9)

    ax1.plot(dates, [date[0][0] for date in stats], color='black')
    ax1.plot(dates, [date[0][1] for date in stats], color='black', linestyle='--')
    ax1.plot(dates, [date[0][2] for date in stats], color='black', linestyle='--')

    ax1.set_xticks(np.arange(0, len(dates), 24))
    ax1.set_title('Market Beta')


    ax2.plot(dates, [date[1][0] for date in stats], color='black')
    ax2.plot(dates, [date[1][1] for date in stats], color='black', linestyle='--')
    ax2.plot(dates, [date[1][2] for date in stats], color='black', linestyle='--')

    ax2.set_xticks(np.arange(0, len(dates), 24))
    ax2.set_title('Interest Rate Beta')


    ax3.plot(dates, [date[2][0] for date in stats], color='black')
    ax3.plot(dates, [date[2][1] for date in stats], color='black', linestyle='--')
    ax3.plot(dates, [date[2][2] for date in stats], color='black', linestyle='--')

    ax3.set_xticks(np.arange(0, len(dates), 24))
    ax3.set_title('Credit Beta')


    ax4.plot(dates, [date[3][0] for date in stats], color='black')
    ax4.plot(dates, [date[3][1] for date in stats], color='black', linestyle='--')
    ax4.plot(dates, [date[3][2] for date in stats], color='black', linestyle='--')

    ax4.set_xticks(np.arange(0, len(dates), 24))
    ax4.set_title('Currency Beta')



    fig.tight_layout()
    
    manager_name = manager_returns.columns[0]
    
    if savefig:
        plt.savefig(f'{manager_name} Macro Factors')

    plt.show()

### The purpose of this notebook is to determine relationships of various investment strategies to macro factors, namely market, interest rate, credit, and commodity risk.

In [2]:
# Input csv file name located on Desktop and the name of the manager
manager_file = 'xgb.csv'

manager_name = 'XGB'

In [4]:
model, summary, start, end = factors.macroOLS(manager_file, start='2010-11')

#factors.exportfactors(summary, manager_name, start, end)

summary

Factor exposures from 2010-11 to 2020-10


<class 'statsmodels.iolib.summary2.Summary'>
"""
                 Results: Ordinary least squares
=================================================================
Model:              OLS              Adj. R-squared:     0.741   
Dependent Variable: XGB              AIC:                221.8777
Date:               2020-11-23 15:30 BIC:                235.8152
No. Observations:   120              Log-Likelihood:     -105.94 
Df Model:           4                F-statistic:        54.77   
Df Residuals:       115              Prob (F-statistic): 9.04e-26
R-squared:          0.750            Scale:              0.35712 
-----------------------------------------------------------------
                   Coef.  Std.Err.    z    P>|z|   [0.025  0.975]
-----------------------------------------------------------------
Alpha             -0.1030   0.0486 -2.1193 0.0341 -0.1983 -0.0077
Market             0.1776   0.0231  7.6820 0.0000  0.1323  0.2229
Interest Rate      0.6263   0.0455 13.7510 0.0000  0.5370  0.7155
Credit             0.2073   0.0324  6.4052 0.0000  0.1439  0.2708
Currency           0.2326   0.0311  7.4765 0.0000  0.1716  0.2936
-----------------------------------------------------------------
Omnibus:              6.474        Durbin-Watson:           2.550
Prob(Omnibus):        0.039        Jarque-Bera (JB):        6.897
Skew:                 0.370        Prob(JB):                0.032
Kurtosis:             3.912        Condition No.:           5    
=================================================================

"""

In [ ]:
rets, macs = factors.macroprep(manager_file)
plot_macro_rolling(rets, macs, savefig=False)

### We want to examine whether making or not making credit/comodity adjustments provides us with the best prediction of next month's return, as well as which ones give us the least correlated errors

In [3]:
def macro_predict(manager_returns, macro_returns, rolling=120, showfig = False, savefig=False):
    
    prediction_errors = []
    dates = []
    rolling = 120

    for i in np.arange(manager_returns.shape[0] - rolling):

        manager_returns_rolling = manager_returns.iloc[i : rolling + i]
        macro_returns_rolling = macro_returns.iloc[i : rolling + i]

        date = manager_returns_rolling.index[-1]

        macro_returns_rolling = sm.tools.add_constant(macro_returns_rolling)
        model = sm.OLS(manager_returns_rolling, macro_returns_rolling).fit(cov_type='HAC', cov_kwds={'maxlags':12})

        summary = model.summary2(xname='Alpha MKT INT CRD CUR'.split()).tables[1]

        mkt = summary.iloc[1][0] * macro_returns.iloc[rolling + i, 0]
        intr = summary.iloc[2][0] * macro_returns.iloc[rolling + i, 1]
        crd = summary.iloc[3][0] * macro_returns.iloc[rolling + i, 2]
        cur = summary.iloc[4][0] * macro_returns.iloc[rolling + i, 3]

        prediction = mkt + intr + crd + cur
        actual = manager_returns.iloc[rolling + i, 0]


        prediction_errors.append(prediction - actual)
        dates.append(date)
    
    mae = np.average(np.abs(prediction_errors))
    rmse = np.sqrt(np.average(np.power(prediction_errors,2)))
    
    print(f'Since {manager_returns.index[0]}')
    print(f'MAE: {mae}')
    print(f'RMSE: {rmse}')
    print(f'Max Error: {np.max(np.abs(prediction_errors))}')
    
    if showfig:
        fig, ax = plt.subplots()
        fig.set_size_inches(10, 5)

        ax.plot(dates, prediction_errors, color='black')

        ax.set_xticks(np.arange(0, len(dates), 24))

        manager_name = manager_returns.columns[0]

        ax.set_title(f'{manager_name} Macro Prediction Errors')


        fig.tight_layout()


        if savefig:
            plt.savefig(f'{manager_name} Macro Prediction Errors')

        plt.show()


    return prediction_errors


In [ ]:
# Input csv file name located on Desktop and the name of the manager
manager_file = 'msci.csv'

manager_name = ''

In [ ]:
rets, macs = factors.macroprep(manager_file)

In [ ]:
predictions = macro_predict(rets, macs)

In [ ]:
macs.to_excel('Macro Returns Since 1993.xlsx')

### We now want to see how correlated errors are between different strategies. This will help us understand relationships when it comes to stress testing.

In [4]:
def macro_errors(manager_returns, macro_returns, rolling=120):
    
    prediction_errors = []
    dates = []
    rolling = 120

    for i in np.arange(manager_returns.shape[0] - rolling):

        manager_returns_rolling = manager_returns.iloc[i : rolling + i]
        macro_returns_rolling = macro_returns.iloc[i : rolling + i]

        date = manager_returns_rolling.index[-1]

        macro_returns_rolling = sm.tools.add_constant(macro_returns_rolling)
        model = sm.OLS(manager_returns_rolling, macro_returns_rolling).fit(cov_type='HAC', cov_kwds={'maxlags':12})

        summary = model.summary2(xname='Alpha MKT INT CRD CUR'.split()).tables[1]

        mkt = summary.iloc[1][0] * macro_returns.iloc[rolling + i, 0]
        intr = summary.iloc[2][0] * macro_returns.iloc[rolling + i, 1]
        crd = summary.iloc[3][0] * macro_returns.iloc[rolling + i, 2]
        cur = summary.iloc[4][0] * macro_returns.iloc[rolling + i, 3]

        prediction = mkt + intr + crd + cur
        actual = manager_returns.iloc[rolling + i, 0]


        prediction_errors.append(prediction - actual)
        dates.append(date)
        
    return prediction_errors

In [ ]:
# Input csv file name located on Desktop and the name of the manager
manager_file = 'tsx.csv'

rets, macs = factors.macroprep(manager_file, start='2001-09')

tsx_errors = macro_errors(rets, macs)

In [ ]:
# Input csv file name located on Desktop and the name of the manager
manager_file = 'msci.csv'

rets, macs = factors.macroprep(manager_file, start='2001-09')

msci_errors = macro_errors(rets, macs)

In [ ]:
# Input csv file name located on Desktop and the name of the manager
manager_file = 'xbb.csv'

rets, macs = factors.macroprep(manager_file, start='2001-09')

xbb_errors = macro_errors(rets, macs)

In [ ]:
np.corrcoef([tsx_errors, msci_errors, xbb_errors])

    